In [20]:
import argparse
import evaluate

from datasets import load_dataset
from fuzzywuzzy import fuzz, process
from rank_bm25 import BM25Okapi
from utils_eval import compute_metrics

rouge = evaluate.load('rouge')

Found cached dataset json (/home/lyf6/.cache/huggingface/datasets/json/default-9ba730f179d68666/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset json (/home/lyf6/.cache/huggingface/datasets/json/default-9a0418f804e277a7/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

## Experiment to Check % of Noise in Original Labels

In [ ]:
# Get dataset from arguments

DATASET_NAME = "cochrane_full"

dataset = load_dataset(
    "json", data_files=f"data/{DATASET_NAME}_multiple.json", field="train"
)
dataset["test"] = load_dataset(
    "json", data_files=f"data/{DATASET_NAME}_multiple.json", field="test"
)["train"]

In [24]:
dataset_labels = [x["labels"][0] for x in dataset["train"]]
dataset_inputs = [x["input"] for x in dataset["train"]]

for idx, item in enumerate(zip(dataset_inputs, dataset_labels)):
    print(f"Idx {idx+1}")
    print(item[0])
    print(item[1])
    print("\n")
    if idx>99:
        break

Idx 1
Two trials met the inclusion criteria. One compared 2% ketanserin ointment in polyethylene glycol (PEG) with PEG alone, used twice a day by 40 participants with arterial leg ulcers, for eight weeks or until healing, whichever was sooner. One compared topical application of blood-derived concentrated growth factor (CGF) with standard dressing (polyurethane film or foam); both applied weekly for six weeks by 61 participants with non-healing ulcers (venous, diabetic arterial, neuropathic, traumatic, or vasculitic). Both trials were small, reported results inadequately, and were of low methodological quality. Short follow-up times (six and eight weeks) meant it would be difficult to capture sufficient healing events to allow us to make comparisons between treatments. One trial demonstrated accelerated wound healing in the ketanserin group compared with the control group. In the trial that compared CGF with standard dressings, the number of participants with diabetic arterial ulcers w

In [22]:
compute_metrics(
    sources = dataset_inputs[:100],
    predictions = dataset_labels[:100],
    labels = [[] for _ in range(100)],
    metrics = ["check_entities"]
)

/home/lyf6/miniconda3/envs/simplification/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/lyf6/miniconda3/envs/simplification/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


{'check_entities': '96/100',
 'check_entities_ents': [['ulcers',
   'second',
   'growth factors',
   'january 2019',
   '49',
   'only two',
   'arterial disease'],
  ['carotid artery narrowing',
   '26 january 2016',
   '47 to 84 years',
   'surgical treatment'],
  ['only three', 'bowel', 'advanced cancer'],
  ['first', 'third', 'second'],
  ['treatments',
   'combined radiotherapy',
   'pleural mesothelioma',
   'up to 21 march 2017',
   'primary pleural cancer'],
  ['blood antibodies',
   '1358',
   'prenatal class',
   '10-minute',
   'third',
   '2790',
   '22.4%',
   'only 5%',
   '2591',
   'toxoplasmosis infection',
   'four',
   '285',
   '13'],
  ['the first two years',
   'two',
   'analysis',
   'injection site reactions',
   'flu-like',
   'only\xa0 ',
   'ifn beta therapy'],
  ['intervention',
   'between 36 and 79',
   'between 65',
   'cvd',
   '4.2 years',
   'harmful',
   'between 457 and 553',
   'only two',
   'july 2019',
   'between 2 years'],
  ['second', 'two']

* XSum: 83/100
* CNN: 74/100
* MedEasi: 80/100
* ASSET: 14/100
* Cochrane: 68/100

## Noisy Examples


In [21]:
dataset_labels[1]

'Two tourist buses have been destroyed by fire in a suspected arson attack in Belfast city centre.'

In [19]:
dataset["train"][1]

{'input': 'A fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST on Saturday and guests were asked to leave the hotel.\nAs they gathered outside they saw the two buses, parked side-by-side in the car park, engulfed by flames.\nOne of the tour groups is from Germany, the other from China and Taiwan. It was their first night in Northern Ireland.\nThe driver of one of the buses said many of the passengers had left personal belongings on board and these had been destroyed.\nBoth groups have organised replacement coaches and will begin their tour of the north coast later than they had planned.\nPolice have appealed for information about the attack.\nInsp David Gibson said: "It appears as though the fire started under one of the buses before spreading to the second.\n"While the exact cause is still under investigation, it is thought that the fire was started deliberately."',
 'labels': ['Two tourist buses have been destroyed by fire in a suspected arson attack in Belfast

## Reordering BRIO for Evaluation

In [4]:
tokenized_corpus = [doc.split(" ") for doc in refs]
bm25 = BM25Okapi(tokenized_corpus)

In [5]:
reordered_preds, reordered_idxs = [], []
for i, s in enumerate(dataset_labels):
    if i%100==0: print(i)
    
    match = bm25.get_top_n(s.split(), refs, n=1)
    # match = process.extractOne(s, refs)
    match_idx = refs_label_2_idx[match[0]]
    pred = preds[match_idx]

    # results = rouge.compute(predictions=[s]*len(refs),
    #                         references=refs,
    #                         use_aggregator=False,
    #                         rouge_types=['rouge1'])
    # match_idx = np.argmax(results["rouge1"])
    # pred = preds[match_idx]

    reordered_preds.append(pred)
    reordered_idxs.append(match_idx)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300


In [6]:
# Write output
with open(f"output/xsum.reordered.txt", "w") as fp:
    for item in reordered_preds:
        fp.write("%s\n" % item)
    print("Done")

Done
